In [1]:
import scipy.io as sio
import numpy as np
# tracks = sio.loadmat('GTSAM_tracks.mat')
# tracks = tracks['results']

'''maintain circular queue for GTSAM oordinates to publish'''
from collections import deque
# ringbuff = deque(tracks, len(tracks))
# a = ringbuff.pop()

In [5]:
#read constraints which will be our factor graphs
# file1 = open("/home/iiith/abhishek/Map-merging/build/data/traj1_traj2_with_inliers.g2o",'r')
file1 = open("/home/iiith/codes/general/3d_isam_merged.g2o",'r')
factors = []
for i in file1:
    if len(i.split())>2:
        values1 = i.split()[1:6]
#         import pdb;pdbD.set_trace()
        prev_frame=int(values1[0])
#         next_frame=int(values1[1])
        x=float(values1[1])
        y=float(values1[2])
        theta =float(values1[3])
        factors.append([x,y,theta])
file1.close()
tracks=np.array(factors)
ringbuff = deque(tracks, len(tracks))

In [2]:
#for 3d coordinated use this code instead of above code
#read constraints which will be our factor graphs
# file1 = open("/home/iiith/abhishek/Map-merging/build/data/traj1_traj2_with_inliers.g2o",'r')
file1 = open("/home/iiith/codes/general/3d_isam_batch.g2o",'r')
factors = []
for i in file1:
    if len(i.split())<=9:
        values1 = i.split()[1:9]
#         import pdb;pdbD.set_trace()
        prev_frame=int(values1[0])
#         next_frame=int(values1[1])
        x=float(values1[1])
        y=float(values1[2])
        z =float(values1[3])
        qx = float(values1[4])
        qy = float(values1[5])
        qz = float(values1[6])
        qw = float(values1[7])
        factors.append([x,y,z,qx,qy,qz,qw])
file1.close()
tracks=np.array(factors)
ringbuff = deque(tracks, len(tracks))

In [3]:
track_3 = tracks[0:1198]
track_1 = tracks[1198:]
track_inverted = np.zeros((1926,7))
track_inverted[0:728] = track_1[0:728]
track_inverted[728:] = track_3[0:]

In [4]:
tracks = track_inverted
ringbuff = deque(tracks, len(tracks))

In [7]:
#!/usr/bin/env python  
import roslib
import math
from nav_msgs.msg import Odometry
import rospy
import tf
from geometry_msgs.msg import Point, Pose, Quaternion, Twist, Vector3
import message_filters
from sensor_msgs.msg import sensor_msgs


pub = rospy.Publisher('/GTSAM/odometry', Odometry, queue_size=10)

def publish_tf(msg1,msg2):
    global ringbuff
    global pub
    pose= ringbuff.popleft()
    br = tf.TransformBroadcaster()
#     x,y,z,roll,pitch,yaw = (pose[0],0,pose[1],0,pose[2],0)
    Tr = np.array([pose[0],pose[1],0])
    vRg = np.array([[1,0,0],[0,0,-1],[0,1,0]])
    Tr = np.matmul(vRg,Tr)
    br.sendTransform((pose[0],pose[1],pose[2]),
                     (pose[3], pose[4],pose[5],pose[6]),
                     msg1.header.stamp,
                     '/zed_tracked_frame',
                     "odom")
    pose_3d = Odometry()
    pose_3d.header.stamp = msg1.header.stamp
    pose_3d.header.frame_id='odom'
    pose_3d.child_frame_id='zed_tracked_frame'
#     angles = tf.transformations.quaternion_from_euler(pose[3], pose[4],pose[5])
    pose_3d.pose.pose = Pose(Point(pose[0],pose[1],pose[2]),Quaternion(pose[3], pose[4],pose[5],pose[6]))
    pub.publish(pose_3d)
    

if __name__ == '__main__':
    rospy.init_node('GTSAM_track')
#     rospy.Subscriber('/stereo_odometer/odometry',
#                      Odometry,
#                      publish_tf)
    I1 = message_filters.Subscriber("/camera/left/image_rect_color", sensor_msgs.msg.Image)
    I2 =  message_filters.Subscriber("/camera/right/image_rect_color", sensor_msgs.msg.Image)
    ts = message_filters.TimeSynchronizer([I1, I2], 10)
    ts.registerCallback(publish_tf)
#     rospy.publish
    rospy.spin()
        

In [1]:
import math
x0, y0= (tracks[0,0],tracks[0,1])
for i in range(1,len(tracks)):
    x1, y1 = (tracks[i,0],tracks[i,1])
    theta= math.atan2(float(y1),float(x1))
#     theta = math.asin(theta)
    tracks[i,2]=theta#+tracks[i-1,2]
#     tracks[i,2] = tracks[i,2] % (math.pi * 2)
    x0, y0 = (x1,y1)
ringbuff = deque(tracks, len(tracks))

NameError: name 'tracks' is not defined

In [2]:
import matplotlib.pyplot as plt
plt.plot(tracks[:,0],tracks[:,1],'o')
plt.show()

NameError: name 'tracks' is not defined

In [5]:
import message_filters
from sensor_msgs.msg import Image, CameraInfo

def callback(image, camera_info):
  # Solve all of perception here...

image_sub = message_filters.Subscriber('image', Image)
info_sub = message_filters.Subscriber('camera_info', CameraInfo)

ts = message_filters.TimeSynchronizer([image_sub, info_sub], 10)
ts.registerCallback(callback)
rospy.spin()

array([  0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
         6.12323400e-17])

In [7]:
tracks = np.array(tracks)
sio.savemat('manual_theta.mat',{'manual_theta_tracks':tracks})


In [27]:
(-math.pi *4.3)  % math.pi *2

4.39822971502571

In [4]:
import numpy

In [7]:
x,y,z=[1,1,1]
pose = np.array([x,y,z])
rot = np.array([[0,1,0],[-1,0,0],[0,0,1]]

In [5]:
import numpy as np

In [5]:
# A = np.array([])
vRg = np.array([[1,0,0],[0,0,-1],[0,1,0]])

In [6]:
Tr = np.array([1,2,3])
Tr = np.matmul(vRg,Tr)

In [8]:
tf.transformations.quaternion_from_euler(1,2,3)

array([-0.71828702,  0.31062245,  0.44443511,  0.43595284])